In [1]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import os

In [2]:
# shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
# os.chdir('/kaggle/working/yolov5') # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.10.0+cu102 _CudaDeviceProperties(name='Tesla V100-SXM2-16GB', major=7, minor=0, total_memory=16160MB, multi_processor_count=80)


In [3]:
os.chdir('/home/ubuntu/vinbigdata/yolov5/')

def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

In [4]:
os.chdir('/home/ubuntu/vinbigdata/yolov5/')

In [5]:
test_df = pd.read_csv('/home/ubuntu/vinbigdata/test.csv')

In [6]:
os.chdir('yolov5/')

In [7]:
fold_exp = ['exp26']
test_dir = f'/home/ubuntu/vinbigdata/DemoConversionToJPG/'

for fold, exp in enumerate(fold_exp):
    weights_dir = f'/home/ubuntu/vinbigdata/yolov5/runs/train/{exp}/weights/best.pt'
    os.chdir('/home/ubuntu/vinbigdata/yolov5/')
    
    !python detect.py --weights $weights_dir\
    --img 1024\
    --conf 0.1\
    --iou 0.4\
    --source $test_dir\
    --save-txt --save-conf --exist-ok
    
    image_ids = []
    PredictionStrings = []

    for file_path in tqdm(glob('runs/detect/exp/labels/*txt')):
        image_id = file_path.split('/')[-1].split('.')[0]
        print(test_df.loc[test_df.file==image_id,['Columns', 'Rows']].values[0])
        w, h = test_df.loc[test_df.file==image_id,['Columns', 'Rows']].values[0]
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
        for idx in range(len(bboxes)):
            bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
        image_ids.append(image_id)
        PredictionStrings.append(' '.join(bboxes))

    # credit / source: https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-infer
    pred_df = pd.DataFrame({'file':image_ids,
                            'PredictionString':PredictionStrings})
    sub_df = pd.merge(test_df, pred_df, on = 'file', how = 'left').fillna("14 1 0 0 1 1")
    sub_df = sub_df[['file', 'PredictionString']]
    sub_df.head()

    os.chdir('/home/ubuntu/vinbigdata/')

    sub_df.to_csv(f'pred/k-fold{fold}-submission.csv',index = False)
    sub_df.tail()
    # remove files and folders from yolov5/runs/detect/exp/
    dir = '/home/ubuntu/vinbigdata/yolov5/runs/detect/exp/'
    for files in os.listdir(dir):
        path = os.path.join(dir, files)
        try:
            shutil.rmtree(path)
        except OSError:
            os.remove(path)

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
detect: weights=['/home/ubuntu/vinbigdata/yolov5/runs/train/exp26/weights/best.pt'], source=/home/ubuntu/vinbigdata/DemoConversionToJPG/, imgsz=[1024, 1024], conf_thres=0.1, iou_thres=0.4, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-115-gbc48457 torch 1.10.0+cu102 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

Fusing layers... 
Model Summary: 444 layers, 86260891 parameters, 0 gradients, 204.2 GFLOPs
image 1/214 /home/ubuntu/vinbigdata/DemoConversionToJPG/002a34c58c5b7

[2521 3408]
[1994 2430]
[1994 2430]
[3072 3072]
[2504 2930]
[2642 3170]
[3072 3072]
[3072 3072]
[3072 3072]
[1994 2430]
[2517 3028]
[2935 3000]
[2271 2877]
[2304 2880]
[2336 2656]
[2346 2665]
[1994 2430]
[2642 3170]
[2022 2389]
[2336 2836]
[1994 2430]
[2642 3170]
[3072 3072]
[2217 2463]
[2336 2836]
[2266 2340]
[2708 3408]
[2322 2721]
[2850 3120]
[2336 2836]
[3000 3000]
[2304 2880]
[2304 2880]
[2316 2600]
[3000 3000]
[1994 2430]
[2145 2706]
[1994 2430]
[2770 2999]
[2180 2420]
[2500 2048]
[1994 2430]
[1994 2430]
[2679 2774]
[1994 2430]
[1994 2430]
[2336 2836]
[3072 3072]
[1994 2430]
[2433 2765]
[1994 2430]
[2304 2880]
[1994 2430]
[3072 3072]
[1994 2430]
[2336 2836]
[1994 2430]
[2642 3170]
[2330 2485]
[3072 3072]
[2448 2862]
[2112 2318]
[2820 3056]
[2948 3000]
[1994 2430]
[3000 3000]
[2368 2936]
[2642 3170]
[1994 2430]
[2815 2940]
[1994 2430]
[2336 2836]
[2048 2500]
[2642 3170]
[3072 3072]
[1994 2430]
[2540 3072]
[2336 2836]
[1994 2430]
[1994 2430]
[1994 2430]
[1994 2430]
[2484 3064]
[199

In [10]:
sub_df

NameError: name 'sub_df' is not defined

In [ ]:
fold_exp = ['exp26', 'exp27', 'exp28', 'exp29', 'exp30']
test_dir = f'/home/ubuntu/vinbigdata/jpg_test/'

for fold, exp in enumerate(fold_exp):
    weights_dir = f'/home/ubuntu/vinbigdata/yolov5/runs/train/{exp}/weights/best.pt'
    os.chdir('/home/ubuntu/vinbigdata/yolov5/')
    
    !python detect.py --weights $weights_dir\
    --img 1024\
    --conf 0.1\
    --iou 0.4\
    --source $test_dir\
    --save-txt --save-conf --exist-ok

    # after detect

    image_ids = []
    PredictionStrings = []

    for file_path in tqdm(glob('runs/detect/exp/labels/*txt')):
        image_id = file_path.split('/')[-1].split('.')[0]
        w, h = test_df.loc[test_df.file==image_id,['Columns', 'Rows']].values[0]
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
        for idx in range(len(bboxes)):
            bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
        image_ids.append(image_id)
        PredictionStrings.append(' '.join(bboxes))

    # credit / source: https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-infer
    pred_df = pd.DataFrame({'file':image_ids,
                            'PredictionString':PredictionStrings})
    sub_df = pd.merge(test_df, pred_df, on = 'file', how = 'left').fillna("14 1 0 0 1 1")
    sub_df = sub_df[['file', 'PredictionString']]
    sub_df.head()

    os.chdir('/home/ubuntu/vinbigdata/')

    sub_df.to_csv(f'pred/k-fold{fold}-submission.csv',index = False)
    sub_df.tail()
    # remove files and folders from yolov5/runs/detect/exp/
    dir = '/home/ubuntu/vinbigdata/yolov5/runs/detect/exp/'
    for files in os.listdir(dir):
        path = os.path.join(dir, files)
        try:
            shutil.rmtree(path)
        except OSError:
            os.remove(path)
    
